In [100]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

data=pd.read_csv('nba.csv',index_col=False)

In [101]:
data=data.drop(['Unnamed: 6','Unnamed: 7','Notes'],axis=1)
data=data.rename(columns={'PTS.1':'PTS_1'})

# 第一个是对阵双方！
# 主客
# 时间
# label是胜负

# 30 features , each feature represents a team, if engage, 1. else 0.
# 1 feature, the previous team is playing at home or away. home 1 else 0.
# playing time. how to represent? 

In [102]:
def who_win(a,b):
    if a > b:
        return 1
    else:
        return 0
data['visitor_win']=data.apply(lambda x: who_win(x.PTS, x.PTS_1), axis = 1)

In [103]:
data.head(10)

,Date,Start (ET),Visitor,PTS,Home,PTS_1,Attend.,v_player1,v_player2,v_player3,v_player4,v_player5,h_player1,h_player2,h_player3,h_player4,h_player5,visitor_win
0,Tue Oct 29 2013,7:00 pm,Orlando Magic,87,Indiana Pacers,97,18165,Arron Afflalo,Nikola Vucevic,Jameer Nelson,Jason Maxiell,Maurice Harkless,Paul George,Lance Stephenson,David West,George Hill,Roy Hibbert,0
1,Tue Oct 29 2013,10:30 pm,Los Angeles Clippers,103,Los Angeles Lakers,116,18997,Blake Griffin,Chris Paul,DeAndre Jordan,J.J. Redick,Jared Dudley,Pau Gasol,Steve Blake,Nick Young,Steve Nash,Shawne Williams,0
2,Tue Oct 29 2013,8:00 pm,Chicago Bulls,95,Miami Heat,107,19964,Derrick Rose,Carlos Boozer,Jimmy Butler,Luol Deng,Joakim Noah,LeBron James,Dwyane Wade,Chris Bosh,Mario Chalmers,Udonis Haslem,0
3,Wed Oct 30 2013,7:00 pm,Brooklyn Nets,94,Cleveland Cavaliers,98,20562,Joe Johnson,Brook Lopez,Paul Pierce,Kevin Garnett,Deron Williams,Tristan Thompson,Anderson Varejao,Kyrie Irving,Dion Waiters,Earl Clark,0
4,Wed Oct 30 2013,8:30 pm,Atlanta Hawks,109,Dallas Mavericks,118,19834,Paul Millsap,Kyle Korver,DeMarre Carroll,Jeff Teague,Al Horford,Monta Ellis,Jose Calderon,Dirk Nowitzki,Shawn Marion,Samuel Dalembert,0
5,Wed Oct 30 2013,7:30 pm,Washington Wizards,102,Detroit Pistons,113,19258,John Wall,Bradley Beal,Trevor Ariza,Nene Hilario,Trevor Booker,Greg Monroe,Josh Smith,Will Bynum,Chauncey Billups,Andre Drummond,0
6,Wed Oct 30 2013,10:30 pm,Los Angeles Lakers,94,Golden State Warriors,125,19596,Steve Blake,Pau Gasol,Jodie Meeks,Nick Young,Shawne Williams,Klay Thompson,Andre Iguodala,David Lee,Stephen Curry,Andrew Bogut,0
7,Wed Oct 30 2013,8:00 pm,Charlotte Bobcats,83,Houston Rockets,96,18083,Kemba Walker,Gerald Henderson,Michael Kidd-Gilchrist,Al Jefferson,Josh McRoberts,Chandler Parsons,James Harden,Dwight Howard,Omer Asik,Patrick Beverley,0
8,Wed Oct 30 2013,8:00 pm,Orlando Magic,115,Minnesota Timberwolves,120,17988,Nikola Vucevic,Arron Afflalo,Jameer Nelson,Maurice Harkless,Jason Maxiell,Kevin Martin,Kevin Love,Ricky Rubio,Corey Brewer,Nikola Pekovic,0
9,Wed Oct 30 2013,8:00 pm,Indiana Pacers,95,New Orleans Pelicans,90,17803,Lance Stephenson,Paul George,George Hill,David West,Roy Hibbert,Anthony Davis,Jrue Holiday,Eric Gordon,Al-Farouq Aminu,Jason Smith,1


In [104]:
team_index={'Atlanta Hawks':0,'Boston Celtics':1,'Brooklyn Nets':2,'Charlotte Hornets':3,'Charlotte Bobcats':3,
           'Chicago Bulls':4,'Cleveland Cavaliers':5,'Dallas Mavericks':6,'Denver Nuggets':7,'Detroit Pistons':8,
           'Golden State Warriors':9,'Houston Rockets':10,'Indiana Pacers':11,'Los Angeles Clippers':12,
           'Los Angeles Lakers':13,'Memphis Grizzlies':14,'Miami Heat':15,'Milwaukee Bucks':16,'Minnesota Timberwolves':17,
           'New Orleans Pelicans':18,'New York Knicks':19,'Oklahoma City Thunder':20,'Orlando Magic':21,'Philadelphia 76ers':22,
           'Phoenix Suns':23,'Portland Trail Blazers':24,'Sacramento Kings':25,'San Antonio Spurs':26,'Toronto Raptors':27,
           'Utah Jazz':28,'Washington Wizards':29}
data_for_baseline=data[['Start (ET)','Visitor','PTS','Home','PTS_1']]
data_for_baseline.head(10)

,Start (ET),Visitor,PTS,Home,PTS_1
0,7:00 pm,Orlando Magic,87,Indiana Pacers,97
1,10:30 pm,Los Angeles Clippers,103,Los Angeles Lakers,116
2,8:00 pm,Chicago Bulls,95,Miami Heat,107
3,7:00 pm,Brooklyn Nets,94,Cleveland Cavaliers,98
4,8:30 pm,Atlanta Hawks,109,Dallas Mavericks,118
5,7:30 pm,Washington Wizards,102,Detroit Pistons,113
6,10:30 pm,Los Angeles Lakers,94,Golden State Warriors,125
7,8:00 pm,Charlotte Bobcats,83,Houston Rockets,96
8,8:00 pm,Orlando Magic,115,Minnesota Timberwolves,120
9,8:00 pm,Indiana Pacers,95,New Orleans Pelicans,90


In [105]:
import numpy as np
y=np.array(data['visitor_win'])
y #label -- 客队是否赢球

array([0, 0, 0, ..., 0, 0, 0])

In [106]:
#30个 0，1 表示参赛球队，1个feature表示在前面的那队是不是客队
X=[]

for index, row in data_for_baseline.iterrows():
    x=np.zeros(31)
    i_visitor=team_index[row['Visitor']]
    i_home=team_index[row['Home']]
    x[i_visitor]=1
    x[i_home]=1
    if i_visitor < i_home:
        x[30]=1
    X.append(x)


In [107]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [108]:
from sklearn.svm import SVC

svm=SVC(random_state=42,probability=True,gamma='auto')
svm.fit(X_train,y_train)
svm.score(X_test,y_test)

0.58265582655826553

In [109]:
career=pd.read_csv('player_career.csv',index_col=False)
pd.set_option('display.max_columns', None)
career = career.dropna(subset=['3P','GS'])
career = career.T
career = career.rename(columns=career.iloc[0]).drop(career.index[0]).dropna(how='all').drop('Lg')

career = career.rename(columns={'Nene': 'Nene Hilario', 'Jeff Taylor':'Jeffery Taylor'})
career=career.fillna(0)

career['Michael Jordan']
# print (career.ix[:, (career == 926).any()])

G        1072
GS       1039
MP       38.3
FG       11.4
FGA      22.9
FG%     0.497
3P        0.5
3PA       1.7
3P%     0.327
2P       10.8
2PA      21.2
2P%      0.51
eFG%    0.509
FT        6.8
FTA       8.2
FT%     0.835
ORB       1.6
DRB       4.7
TRB       6.2
AST       5.3
STL       2.3
BLK       0.8
TOV       2.7
PF        2.6
PTS      30.1
Name: Michael Jordan, dtype: object

In [110]:
career

Alaa Abdelnaby Kareem Abdul-Jabbar Mahmoud Abdul-Rauf Tariq Abdul-Wahad  \
G               256                1560                586               236   
GS               53                 625                336               145   
MP             12.5                36.8               26.7              20.4   
FG              2.4                10.2                  6               3.1   
FGA             4.8                18.1               13.6               7.3   
FG%           0.502               0.559              0.442             0.417   
3P                0                   0                0.8               0.1   
3PA               0                   0                2.3               0.3   
3P%               0               0.056              0.354             0.237   
2P              2.4                10.2                5.2                 3   
2PA             4.8                18.1               11.3                 7   
2P%           0.503                0.56               0.46             0.425   
eFG%          0.502               0.559              0.472             0.422   
FT              0.9                 4.3                1.8               1.6   
FTA             1.3                   6                  2               2.2   
FT%           0.701               0.721              0.905             0.703   
ORB             1.1                 2.4                0.4               1.2   
DRB             2.2                 7.6                1.5               2.1   
TRB             3.3                11.2                1.9               3.3   
AST             0.3                 3.6                3.5               1.1   
STL             0.3                 0.9                0.8               0.8   
BLK             0.3                 2.6                0.1               0.4   
TOV             1.0                 2.7                1.6               1.3   
PF              1.9                   3                1.9               2.1   
PTS             5.7                24.6               14.6               7.8   

     Shareef Abdur-Rahim Alex Abrines Alex Acker Mark Acres Quincy Acy  \
G                    830          163         30        375        327   
GS                   704           16          0         68         60   
MP                  34.8         15.9        7.8         16       16.1   
FG                   6.5          1.8        1.1        1.4        1.8   
FGA                 13.9          4.6        3.1        2.7          4   
FG%                0.472        0.387       0.37      0.506      0.447   
3P                   0.2          1.3        0.3          0        0.6   
3PA                  0.6          3.4        0.8          0        1.6   
3P%                0.297        0.371       0.32      0.538      0.356   
2P                   6.4          0.5        0.9        1.4        1.2   
2PA                 13.2          1.1        2.2        2.7        2.4   
2P%                 0.48        0.433      0.388      0.505      0.505   
eFG%               0.479        0.526      0.413      0.509      0.517   
FT                   4.8          0.6        0.2        0.8        0.9   
FTA                    6          0.6        0.3        1.2        1.1   
FT%                 0.81        0.875        0.5      0.665      0.761   
ORB                  2.3          0.3        0.3        1.5        0.9   
DRB                  5.3          1.1        0.7        2.5        2.6   
TRB                  7.5          1.4          1        4.1        3.5   
AST                  2.5          0.5        0.5        0.5        0.6   
STL                    1          0.5        0.2        0.4        0.4   
BLK                  0.8          0.1        0.1        0.3        0.4   
TOV                  2.6          0.4        0.4        0.6        0.7   
PF                   2.8          1.7        0.4        2.5          2   
PTS                 18.1          5.4        2.7        3.6          5   


In [111]:
import numpy as np
def feature_prep(game_df, player_df):
    
    y_train = np.array(game_df['visitor_win'])
    
    stat_names = np.array(player_df.index.values)
    
    x_train = []
    for index, row in game_df.iterrows():
#         feature_dict = {}
        feature = []
        names = row[7:17]
        for name in names:
            if name == 'Luigi Datome': name = 'Gigi Datome'
            if name == 'Kelly Oubre': name = 'Kelly Oubre Jr.'
            if name == 'Derrick Jones': name = 'Derrick Jones Jr.'
            if name == 'Taurean Waller-Prince': name = 'Taurean Prince'
            if name == 'Dennis Smith': name = 'Dennis Smith Jr.'
            if name == 'Frank Mason': name = 'Frank Mason III'
            player_stat = player_df[name].values
            temp = [stat for stat in player_stat]
            #print(name,temp)
            feature.extend(temp) 
        feature = np.array(feature)
        x_train.append(feature)
#             feature_dict[name] = {}
#             player_stat = player_df[name].values
#             for i in range(len(player_stat)):
#                 feature_dict[name][stat_names[i]] = player_stat[i]
#         x_train.append(feature_dict)
    
    #print(x_train[0])
    
    return (np.array(x_train), np.array(y_train))                       


In [112]:
X, y = feature_prep(data, career)

In [123]:
for i in range(len(X)):
    array = np.array(X[i], dtype=np.float64, order='C')

In [115]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.3,random_state=42)
print (X_train.shape)
print (y_train.shape)
svm=SVC(random_state=42)
svm.fit(X_train,y_train)
svm.score(X_test,y_test)


(4305, 250)
(4305,)


0.58265582655826553

In [116]:
#What if Kobe Bryant still plays for Lakers?

#2016-2017 season
LA_2016=data[(data.Home=='Los Angeles Lakers') | (data.Visitor=='Los Angeles Lakers')][246:328]

def switch(x):
    for i in range(len(x)):
        if x[i]=='Nick Young' or x[i]=='Lou Williams' or x[i]=='David Nwaba' or x[i]=='Jordan Clarkson': # 4 SG in Lakers roster that year
            x[i]='Kobe Bryant'
            break
    return x
LA_2016[['v_player1','v_player2','v_player3','v_player4','v_player5',
       'h_player1','h_player2','h_player3','h_player4','h_player5']]=LA_2016.apply(lambda x: switch(x[7:17]),axis=1)

x_predict,y_useless=feature_prep(LA_2016,career)
y_predict=svm.predict(x_predict)

visitor=list(LA_2016[['Visitor']].values)
win=0
for i in range(len(visitor)):
    if visitor[i][0]=='Los Angeles Lakers' and y_predict[i]==1:
        win+=1
    elif visitor[i][0]!='Los Angeles Lakers' and y_predict[i]==0:
        win+=1
        
def count_app(x):
    count=0
    if x=='Kobe Bryant':
        count+=1
    return count
app=np.sum(np.array(LA_2016.applymap(count_app)))        
        
print('Lakers could get %d wins, %d losts with Kobe Bryant starting %d games in 2016-2017 season'%(win,82-win,app))
print('While they actually got 26 wins, 56 losts in that season')

Lakers could get 41 wins, 41 losts with Kobe Bryant starting 78 games in 2016-2017 season
While they actually got 26 wins, 56 losts in that season


In [117]:
LA_2016

,Date,Start (ET),Visitor,PTS,Home,PTS_1,Attend.,v_player1,v_player2,v_player3,v_player4,v_player5,h_player1,h_player2,h_player3,h_player4,h_player5,visitor_win
3695,Wed Oct 26 2016,10:30 pm,Houston Rockets,114,Los Angeles Lakers,120,18997,James Harden,Eric Gordon,Ryan Anderson,Trevor Ariza,Clint Capela,Julius Randle,D'Angelo Russell,Luol Deng,Kobe Bryant,Timofey Mozgov,0
3714,Fri Oct 28 2016,9:00 pm,Los Angeles Lakers,89,Utah Jazz,96,19911,Luol Deng,D'Angelo Russell,Julius Randle,Kobe Bryant,Timofey Mozgov,George Hill,Rodney Hood,Rudy Gobert,Joe Johnson,Boris Diaw,0
3729,Sun Oct 30 2016,7:00 pm,Los Angeles Lakers,96,Oklahoma City Thunder,113,18203,Julius Randle,D'Angelo Russell,Kobe Bryant,Timofey Mozgov,Luol Deng,Victor Oladipo,Russell Westbrook,Andre Roberson,Steven Adams,Domantas Sabonis,0
3737,Tue Nov 1 2016,7:00 pm,Los Angeles Lakers,108,Indiana Pacers,115,15348,Kobe Bryant,Luol Deng,Julius Randle,Timofey Mozgov,D'Angelo Russell,Monta Ellis,Paul George,Jeff Teague,Thaddeus Young,Myles Turner,0
3744,Wed Nov 2 2016,7:30 pm,Los Angeles Lakers,123,Atlanta Hawks,116,13800,D'Angelo Russell,Julius Randle,Luol Deng,Kobe Bryant,Ivica Zubac,Paul Millsap,Dwight Howard,Kent Bazemore,Dennis Schroder,Kyle Korver,1
3762,Fri Nov 4 2016,10:30 pm,Golden State Warriors,97,Los Angeles Lakers,117,18997,Klay Thompson,Kevin Durant,Draymond Green,Stephen Curry,Zaza Pachulia,Julius Randle,Kobe Bryant,D'Angelo Russell,Luol Deng,Timofey Mozgov,0
3778,Sun Nov 6 2016,9:30 pm,Phoenix Suns,108,Los Angeles Lakers,119,18997,Devin Booker,T.J. Warren,Tyson Chandler,Eric Bledsoe,Jared Dudley,Kobe Bryant,Julius Randle,D'Angelo Russell,Timofey Mozgov,Luol Deng,0
3791,Tue Nov 8 2016,10:30 pm,Dallas Mavericks,109,Los Angeles Lakers,97,18997,Harrison Barnes,Wesley Matthews,J.J. Barea,Andrew Bogut,Dorian Finney-Smith,Julius Randle,Luol Deng,Kobe Bryant,D'Angelo Russell,Timofey Mozgov,1
3809,Thu Nov 10 2016,10:30 pm,Los Angeles Lakers,101,Sacramento Kings,91,17608,Timofey Mozgov,Kobe Bryant,Julius Randle,D'Angelo Russell,Luol Deng,DeMarcus Cousins,Rudy Gay,Arron Afflalo,Ty Lawson,Kosta Koufos,1
3826,Sat Nov 12 2016,7:00 pm,Los Angeles Lakers,126,New Orleans Pelicans,99,17138,D'Angelo Russell,Julius Randle,Timofey Mozgov,Kobe Bryant,Luol Deng,Anthony Davis,Tim Frazier,E'Twaun Moore,Dante Cunningham,Omer Asik,1


In [118]:
#2017-2018 season
LA_2017=data[(data.Home=='Los Angeles Lakers') | (data.Visitor=='Los Angeles Lakers')][328:410]

def switch(x):
    for i in range(len(x)):
        if x[i]=='Vander Blue' or x[i]=='Kentavious Caldwell-Pope' or x[i]=='Josh Hart' or x[i]=='Jordan Clarkson' or x[i]=='Andre Ingram': # 4 SG in Lakers roster that year
            x[i]='Kobe Bryant'
            break
    return x
LA_2017[['v_player1','v_player2','v_player3','v_player4','v_player5',
       'h_player1','h_player2','h_player3','h_player4','h_player5']]=LA_2017.apply(lambda x: switch(x[7:17]),axis=1)

x_predict,y_useless=feature_prep(LA_2017,career)
y_predict=svm.predict(x_predict)

visitor=list(LA_2017[['Visitor']].values)
win=0
for i in range(len(visitor)):
    if visitor[i][0]=='Los Angeles Lakers' and y_predict[i]==1:
        win+=1
    elif visitor[i][0]!='Los Angeles Lakers' and y_predict[i]==0:
        win+=1
        
def count_app(x):
    count=0
    if x=='Kobe Bryant':
        count+=1
    return count
app=np.sum(np.array(LA_2017.applymap(count_app)))        
        
print('Lakers could get %d wins, %d losts with Kobe Bryant starting %d games in 2017-2018 season'%(win,82-win,app))
print('While they actually got 35 wins, 47 losts in that season')

Lakers could get 41 wins, 41 losts with Kobe Bryant starting 80 games in 2017-2018 season
While they actually got 35 wins, 47 losts in that season


In [119]:
#What if Michael Jordan's Bulls plays in today's league?
#What if Draymond Green hasn't got injured in 2015-2016's finals?
#What if T-mac and Yao's Rockets plays in today's league?
#Is Carmelo Anthony really unsuitable for Rockets?
#What if a team owns 5 Lebron James?
#Can Golden State Warriors defeat Cavaliers these years without Kevin Durant?
#What will be the final ranking of this regular season?

In [122]:
#What if Michael Jordan's Bulls plays in today's league?

chi_2017=data[(data.Home=='Chicago Bulls') | (data.Visitor=='Chicago Bulls')][246:328]

def back_to_1996(x):
    if x[2] == 'Chicago Bulls': 
        x[7],x[8],x[9],x[10],x[11]='Michael Jordan','Scottie Pippen','Dennis Rodman','Luc Longley','Ron Harper'
    if x[4] == 'Chicago Bulls': 
        x[12],x[13],x[14],x[15],x[16]='Michael Jordan','Scottie Pippen','Dennis Rodman','Luc Longley','Ron Harper'
    return x
chi_2017[['Date','Start (ET)','Visitor','PTS','Home','PTS_1','Attend.','v_player1','v_player2','v_player3','v_player4','v_player5',
       'h_player1','h_player2','h_player3','h_player4','h_player5']]=chi_2017.apply(lambda x: back_to_1996(x[0:17]),axis=1)


x_predict,y_useless=feature_prep(chi_2017,career)
y_predict=svm.predict(x_predict)

visitor=list(chi_2017[['Visitor']].values)
win=0
for i in range(len(visitor)):
    if visitor[i][0]=='Chicago Bulls' and y_predict[i]==1:
        win+=1
    elif visitor[i][0]!='Chicago Bulls' and y_predict[i]==0:
        win+=1

print('Chicago Bulls could get %d wins, %d losts in 2016-2017 season with their starting fives back in 1995-1996 season'%(win,82-win))
print('While they actually got 72 wins, 10 losts back in 1995-1996')



Chicago Bulls could get 41 wins, 41 losts in 2016-2017 season with their starting fives back in 1995-1996 season
While they actually got 72 wins, 10 losts back in 1995-1996


In [121]:
chi_2017

,Date,Start (ET),Visitor,PTS,Home,PTS_1,Attend.,v_player1,v_player2,v_player3,v_player4,v_player5,h_player1,h_player2,h_player3,h_player4,h_player5,visitor_win
3704,Thu Oct 27 2016,8:00 pm,Boston Celtics,99,Chicago Bulls,105,21501,Avery Bradley,Jae Crowder,Isaiah Thomas,Al Horford,Amir Johnson,Michael Jordan,Scottie Pippen,Dennis Rodman,Luc Longley,Ron Harper,0
3715,Sat Oct 29 2016,8:00 pm,Indiana Pacers,101,Chicago Bulls,118,21373,Jeff Teague,Monta Ellis,Thaddeus Young,Paul George,Myles Turner,Michael Jordan,Scottie Pippen,Dennis Rodman,Luc Longley,Ron Harper,0
3732,Mon Oct 31 2016,7:30 pm,Chicago Bulls,118,Brooklyn Nets,88,15842,Michael Jordan,Scottie Pippen,Dennis Rodman,Luc Longley,Ron Harper,Rondae Hollis-Jefferson,Jeremy Lin,Brook Lopez,Bojan Bogdanovic,Trevor Booker,1
3745,Wed Nov 2 2016,8:00 pm,Chicago Bulls,100,Boston Celtics,107,18624,Michael Jordan,Scottie Pippen,Dennis Rodman,Luc Longley,Ron Harper,Avery Bradley,Isaiah Thomas,Amir Johnson,Tyler Zeller,Jae Crowder,0
3760,Fri Nov 4 2016,8:00 pm,New York Knicks,117,Chicago Bulls,104,22376,Courtney Lee,Derrick Rose,Carmelo Anthony,Joakim Noah,Kristaps Porzingis,Michael Jordan,Scottie Pippen,Dennis Rodman,Luc Longley,Ron Harper,1
3770,Sat Nov 5 2016,7:00 pm,Chicago Bulls,94,Indiana Pacers,111,17020,Michael Jordan,Scottie Pippen,Dennis Rodman,Luc Longley,Ron Harper,Monta Ellis,Paul George,Jeff Teague,Myles Turner,Thaddeus Young,0
3782,Mon Nov 7 2016,8:00 pm,Orlando Magic,80,Chicago Bulls,112,21320,Aaron Gordon,Evan Fournier,Serge Ibaka,Nikola Vucevic,Elfrid Payton,Michael Jordan,Scottie Pippen,Dennis Rodman,Luc Longley,Ron Harper,0
3795,Wed Nov 9 2016,7:30 pm,Chicago Bulls,107,Atlanta Hawks,115,16354,Michael Jordan,Scottie Pippen,Dennis Rodman,Luc Longley,Ron Harper,Paul Millsap,Dwight Howard,Dennis Schroder,Kyle Korver,Kent Bazemore,0
3807,Thu Nov 10 2016,8:00 pm,Chicago Bulls,98,Miami Heat,95,19600,Michael Jordan,Scottie Pippen,Dennis Rodman,Luc Longley,Ron Harper,Justise Winslow,Hassan Whiteside,Dion Waiters,Goran Dragic,Luke Babbitt,1
3819,Sat Nov 12 2016,8:00 pm,Washington Wizards,95,Chicago Bulls,106,21962,Markieff Morris,Sheldon Mac,Marcin Gortat,Tomas Satoransky,Otto Porter,Michael Jordan,Scottie Pippen,Dennis Rodman,Luc Longley,Ron Harper,0
